In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains as Actions

In [2]:
def HouseDF(houseLines):

    "retrieves details of the houses into a table"

    columns=["Category(sale/Rent/Sold)", "House Features", "Address", "State",
                               "Zip code", "Price", "Open Time/Posting Time"]

    houseTable={x:[] for x in columns}    

    iD=1

    for house in houseLines:

        print()

        print("Index:",iD)

        print("House:",house)

        iD+=1

        category=house[3].split("-")[-1]

        features=house[3].split("-")[0]

        address=house[0]

        state=house[0].split(",")[1]

        zipCode=house[0].split(",")[-1]

        price=house[2]


        if "$" not in price:

            price=house[1]

            category=house[2].split("-")[-1]

            features=house[2].split("-")[0]

        if len(house)==5:

            timings=house[4]
        
        else:

            timings="Not Available"

        houseTable[columns[0]].append(category)

        houseTable[columns[1]].append(features)

        houseTable[columns[2]].append(address)

        houseTable[columns[3]].append(state)

        houseTable[columns[4]].append(zipCode)

        houseTable[columns[5]].append(price)

        houseTable[columns[6]].append(timings)
    

    return pd.DataFrame(houseTable)

        

In [3]:
def CollectHouseAllPages():

    "collects all details of houses from the link provided below"

    driver = webdriver.Chrome(service=Service("chromedriver.exe"))

    # Web Page Featuring Houses

    #webPage=driver.get('https://www.zillow.com/homes/')

    webPage=driver.get("https://www.zillow.com/co/?searchQueryState")

    driver.maximize_window()

    # Collect houses from all pages

    houseLinesAllPages=[]

    allPagesExtracted=False

    pageNum=1

    while allPagesExtracted==False:

        try:

            houses = WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div[5]/div/div/div/div[1]/ul"))
            )        

        except Exception:

            print("Unable to extract any details from the page. Please make sure houses are loaded on to the screen")

            break

        houseList=driver.find_elements(By.XPATH,"//article[@role='presentation']")
        
        houseLines=[x.text.split("\n") for x in houseList]

        print('houseLines:',houseLines)

        houseLinesAllPages+=houseLines

        nextPageIcon = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH,"//a[@title='Next page']"))
            ) 
        
        disabled=nextPageIcon.get_attribute("aria-disabled")

        print("Next Page Details:",disabled)

        if disabled=="true":

            allPagesExtracted=True
        
        else:

            webdriver.ActionChains(driver).move_to_element(nextPageIcon ).click(nextPageIcon ).perform()

        pageNum+=1
    

    print("houses total:",houseLinesAllPages)

    houseDetails=HouseDF(houseLinesAllPages)

    return houseDetails

In [4]:
houseAll=CollectHouseAllPages()

houseAll.to_csv("house_extracted.csv")

houseLines: [['1600 Berkley Ave, Pueblo, CO 81004', 'WEDGEWOOD HOMES REALTY. Listing provided by PAR', '$270,000', '3 bds2 ba1,052 sqft - House for sale', '3D Tour', 'Image 1 of 28'], ['18112 E Crestline Circle, Centennial, CO 80015', 'MLS ID #7817541, RE/MAX ALLIANCE', '$469,900', '3 bds3 ba1,880 sqft - House for sale', '3D Tour', 'Image 1 of 34'], ['2899 W 10th Avenue, Denver, CO 80204', 'MLS ID #3334981, INVALESCO REAL ESTATE', '$312,000', '3 bds2 ba1,648 sqft - Townhouse for sale', '27 days on Zillow', 'Image 1 of 14'], ['9365 Daisy Court, Highlands Ranch, CO 80126', 'MLS ID #5312168, CJV REAL ESTATE', '$549,990', '3 bds2 ba1,314 sqft - House for sale', '3D Tour', 'Image 1 of 39'], ['1200 N Emerson Street Unit 301, Denver, CO 80218', 'MLS ID #9635319, YOUR CASTLE REALTY LLC', '$395,000', '2 bds2 ba1,032 sqft - Condo for sale', '3D Tour', 'Image 1 of 12'], ['9932 Fairwood Street, Littleton, CO 80125', 'MLS ID #9853536, ORCHARD BROKERAGE LLC', '$550,000', '3 bds3 ba2,264 sqft - House

In [5]:
houseAll

,Category(sale/Rent/Sold),House Features,Address,State,Zip code,Price,Open Time/Posting Time
0,House for sale,"3 bds2 ba1,052 sqft","1600 Berkley Ave, Pueblo, CO 81004",Pueblo,CO 81004,"$270,000",Not Available
1,House for sale,"3 bds3 ba1,880 sqft","18112 E Crestline Circle, Centennial, CO 80015",Centennial,CO 80015,"$469,900",Not Available
2,Townhouse for sale,"3 bds2 ba1,648 sqft","2899 W 10th Avenue, Denver, CO 80204",Denver,CO 80204,"$312,000",Not Available
3,House for sale,"3 bds2 ba1,314 sqft","9365 Daisy Court, Highlands Ranch, CO 80126",Highlands Ranch,CO 80126,"$549,990",Not Available
4,Condo for sale,"2 bds2 ba1,032 sqft","1200 N Emerson Street Unit 301, Denver, CO 80218",Denver,CO 80218,"$395,000",Not Available
...,...,...,...,...,...,...,...
795,House for sale,"6 bds4 ba3,586 sqft","12961 E Cedar Avenue, Aurora, CO 80012",Aurora,CO 80012,"$599,000",Not Available
796,House for sale,"3 bds2 ba1,116 sqft","1947 Euclid Ave, Pueblo, CO 81004",Pueblo,CO 81004,"$265,000",Not Available
797,House for sale,"3 bds3 ba2,510 sqft","9261 W Tennessee Avenue, Lakewood, CO 80226",Lakewood,CO 80226,"$495,000",Not Available
798,House for sale,"3 bds3 ba3,039 sqft","1124 Roush Dr, Pagosa Springs, CO 81147",Pagosa Springs,CO 81147,"$975,000",Image 1 of 35
